# Preprocessing (Brilliant Diamonds)

Von *scikit-learn* benötigen wir:

- ``compose`` für Transformation unterschiedlicher Spaltentypen
- ``preprocessing`` für die eigentlichen Transformatoren
- ``tree`` und ``metrics`` für die Testklassifikation

In [ ]:
import pandas as pd
import numpy as np
import brilliantdia_iohelper
from sklearn import compose, preprocessing, tree, metrics

Einlesen der Trainings- und Testdaten.

In [ ]:
dfTrain, boundaries = brilliantdia_iohelper.readAsClass('brilliantdia-train.csv', 4)
dfTest, _ = brilliantdia_iohelper.readAsClass('brilliantdia-test.csv', 4)
dfTrain.info()

Prüfen, ob Werte fehlen (durch einfaches Zählen aller Fehlwerte).

In [ ]:
np.count_nonzero(dfTrain.isna())

Auswahl der numerischen und kategorialen Spalten. ``ityp`` nimmt nur zwei Werte an, keine Konvertierung erforderlich.

In [ ]:
numCols = ['carat', 'icol', 'icut', 'iclar']
catCols = ['ishape', 'irep']
passCols = ['ityp']

Anzeige der unterschiedlichen Werte je Kategorie.

In [ ]:
for col in catCols:
    print('Train:', col, dfTrain[col].unique())
    print('Test', col, dfTest[col].unique())

Dichteplot der metrischen Spalten.

In [ ]:
ax = dfTrain[numCols].plot.density(subplots=True)

… und ein Boxplot.

In [ ]:
ax = dfTrain[numCols].plot.box()

Erzeugen der Transformation. Jeder Transformer ist ein Tripel aus Name, Transformerobjekt und Spaltenliste.
Nicht aufgeführte Spalten werden ignoriert.

- Training darf nur mit den Trainingsdaten erfolgen.
- Transformation sowohl für Trainings- als auch für Testdaten.

Hinweis: Sollen Daten mit unbekannten Labels später klassifiziert werden, muss die Zielspalte vor dem Training
und der Transformation von den Trainings- und Testdaten entfernt werden.

In [ ]:
tr = compose.ColumnTransformer([
    #('num', preprocessing.MinMaxScaler(), numCols),
    ('num', preprocessing.StandardScaler(), numCols),
    ('cat', preprocessing.OneHotEncoder(), catCols),
    ('pass', 'passthrough', passCols),
])
Xtrain = tr.fit(dfTrain).transform(dfTrain)
Xtest = tr.transform(dfTest)
dfTrain.shape, Xtrain.shape

Anzeige der transformierten Daten.

In [ ]:
np.around(Xtrain[:5, :], 2)

Erzeugen eines Data Frame aus den transformierten Daten für die statistische Auswertung.

In [ ]:
dfNumTrain = pd.DataFrame(Xtrain[:, :4], columns=dfTrain.columns[:4])
dfNumTest = pd.DataFrame(Xtest[:, :4], columns=dfTest.columns[:4])

Density Plot und Boxplot der transformierten Trainingsdaten.

In [ ]:
ax = dfNumTrain.plot.density(subplots=True)

In [ ]:
ax = dfNumTrain.plot.box()

Das gleiche für die Testdaten.

In [ ]:
ax = dfNumTest.plot.density(subplots=True)

In [ ]:
ax = dfNumTest.plot.box()

Erzeugen der Zielvektoren, Prüfen der Dimensionen.

In [ ]:
yTrain = dfTrain.priceClass.values
yTest = dfTest.priceClass.values
Xtrain.shape, yTrain.shape, Xtest.shape, yTest.shape

Erzeugen, Trainieren und Bewerten eines Entscheidungsbaums.

In [ ]:
classifier = tree.DecisionTreeClassifier()
classifier.fit(Xtrain, yTrain)
print('Train score', classifier.score(Xtrain, yTrain))
print('Test score', classifier.score(Xtest, yTest))

Confusion Matrix zur detailierten Bewertung.

In [ ]:
predTest = classifier.predict(Xtest)
metrics.confusion_matrix(yTest, predTest)